In [1]:
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import mlp_builder as bare_model_builder

PWD = '../../..'
sys.path.append(PWD)

In [2]:
PROJECT_NAME = 'mlp'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 4096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22134, 24), (17707, 24), (4427, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                450       
                                                                 
 dense_1 (Dense)             (None, 12)                228       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 44s]
val_loss: 9.130734920501709

Best val_loss So Far: 8.731753826141357
Total elapsed time: 00h 22m 07s


# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 4, 'units_mult': 2, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1024
========== Model 1 ==========
{'n_hidden_layers': 8, 'units_mult': 8, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 4921
========== Model 2 ==========
{'n_hidden_layers': 2, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 11905
========== Model 3 ==========
{'n_hidden_layers': 1, 'units_mult': 32, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.5, 'normalize_signal': False}
Number of parameters: 2593
========== Model 4 ==========
{'n_hidden_layers': 1, 'units_m

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME, n_epochs=N_EPOCHS,
                                           batch_size=BATCH_SIZE, n_top=TOP_N, es_min_delta=ES_MIN_DELTA,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 4, 'units_mult': 2, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1024
Got score: 8.7816 (8.7738, 8.7893)
Got score: 12.5676 (8.1117, 17.0234)
Got score: 8.3489 (8.3304, 8.3675)
Got score: 8.4070 (8.4334, 8.3807)
Got score: 8.2212 (8.2163, 8.2260)


{'n_hidden_layers': 8, 'units_mult': 8, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 4921
Got score: 8.8380 (8.8541, 8.8220)
Got score: 8.2749 (8.1400, 8.4097)
Got score: 8.3534 (8.2851, 8.4216)
Got score: 8.3735 (8.3498, 8.3973)
Got score: 8.1996 (8.1422, 8.2571)


{'n_hidden_layers': 2, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 11905
Got score: 8.8411 (8.8270, 8.8552)
Got score: 8.3423 (8.1806, 8.5039)
Got score: 8.3594 (8.3985, 8.3204)
Got score: 8.6968 (8.4595, 8.9342)
Got score: 8.2570 (8.2806, 8.2334)


{'n_hidden_layers': 1, 'units_mult': 32, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.5, 'normalize_signal': False}
Number of parameters: 2593
Got score: 8.8285 (8.8464, 8.8107)
Got score: 8.1568 (8.1012, 8.2124)
Got score: 8.2924 (8.2679, 8.3169)
Got score: 8.3917 (8.4794, 8.3041)
Got score: 8.1221 (8.1389, 8.1053)


{'n_hidden_layers': 1, 'units_mult': 2, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 253
Got score: 9.3371 (9.2872, 9.3870)
Got score: 8.2711 (8.2898, 8.2524)
Got score: 8.4827 (8.4421, 8.5234)
Got score: 8.4780 (8.4644, 8.4916)
Got score: 8.3883 (8.4007, 8.3759)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,9.27,1.66,1024
1,8.41,0.22,4921
2,8.50,0.23,11905
3,8.36,0.25,2593
4,8.59,0.38,253
